In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import uproot
import csv

In [2]:
hits = uproot.open("string_0.root")["string_tree"]
hits = hits.arrays(library="pd")
hits['wavelength'] = 1240/hits['e0']
hits['EventId'] = hits['EventId'].apply(lambda x: x[0])
hits.set_index('EventId', inplace=True)

In [3]:
qe_data = pd.read_csv('../qe.csv')
qe = interp1d(qe_data['wavelength'], qe_data['qe'], kind='cubic', fill_value='extrapolate')

In [4]:
hits['qe'] = hits['wavelength'].apply(lambda x: np.random.random(len(x)) < qe(x)/100)

In [5]:
hits['t0'] = hits.apply(lambda row: [val for i, val in enumerate(row['t0']) if row['qe'][i]], axis=1)
hits['e0'] = hits.apply(lambda row: [val for i, val in enumerate(row['e0']) if row['qe'][i]], axis=1)
hits['wavelength'] = hits.apply(lambda row: [val for i, val in enumerate(row['wavelength']) if row['qe'][i]], axis=1)
hits['PmtId'] = hits.apply(lambda row: [val for i, val in enumerate(row['PmtId']) if row['qe'][i]], axis=1)
hits['DomId'] = hits.apply(lambda row: [val for i, val in enumerate(row['DomId']) if row['qe'][i]], axis=1)
hits = hits.drop('qe',axis=1)


In [6]:
hits = hits[hits['wavelength'].apply(len) > 0]

In [7]:
muon = pd.read_json('../muon.json')

In [8]:
hits.loc[:, 'weights'] = muon.loc[muon['event_id'].isin(hits.index), 'weights'].apply(lambda x: x.get('spectrum'))

In [9]:
# 创建一个新的列，表示weights值是否和上一行的值相同
hits.loc[:,'weights_shifted'] = hits['weights'].shift()
hits['weights_same_as_previous'] = hits['weights'] == hits['weights_shifted']

# 根据新的列对数据进行分组，并将每一组的数据合并到一起
hits['group'] = ((hits['weights_same_as_previous']==False)).cumsum()
merged_hits = hits
# # 删除不再需要的列
merged_hits = merged_hits.drop(['weights_shifted', 'weights_same_as_previous'], axis=1)

In [10]:
merged_hits.set_index('group', inplace=True)

In [11]:
time_window = 20
domhitfile = open('domhits.csv', 'a')
max_group = merged_hits.index.max()
for i in range(1, max_group+1):
    pmthits = np.zeros((21,31,2000))
    time_group = np.array(merged_hits.loc[i]['t0']).ravel().flatten()
    pmt_group = np.array(merged_hits.loc[i]['PmtId']).ravel().flatten()
    dom_group = np.array(merged_hits.loc[i]['DomId']).ravel().flatten()
    weights = np.array(merged_hits.loc[i]['weights']).ravel()[0]
    
  
    time_group = np.hstack(time_group)
    pmt_group = np.hstack(pmt_group)
    dom_group = np.hstack(dom_group)

    time_group = time_group - min(time_group)
    time_group = time_group//time_window

    dom_group = dom_group.astype(int)
    pmt_group = pmt_group.astype(int)
    time_group = time_group.astype(int)

    for j in range(len(time_group)):
        if time_group[j] < 2000:
            pmthits[dom_group[j]][pmt_group[j]][time_group[j]] = 1
            
    domhits = pmthits.sum(axis=1)
    for k in range(21):
        for l in range(1,21):
            count = np.count_nonzero(domhits[k] == l)
            domhitfile.write(str(count))
            domhitfile.write(',')
        domhitfile.write(str(weights))
        domhitfile.write('\n')
    
domhitfile.close()

    

In [12]:
merged_hits

,t0,e0,PmtId,DomId,wavelength,weights
group,,,,,,
1,"[492590.15625, 492919.59375, 492550.5625]","[2.8281500339508057, 2.4775631427764893, 2.951...","[1.0, 24.0, 6.0]","[1.0, 4.0, 1.0]","[438.44915771484375, 500.4917907714844, 420.07...",4.103634e-07
2,"[400405.1875, 400718.625, 400322.625]","[2.486269950866699, 2.658818006515503, 2.98282...","[4.0, 3.0, 22.0]","[11.0, 14.0, 10.0]","[498.73907470703125, 466.3726501464844, 415.71...",1.600684e-07
3,"[605626.75, 605617.875, 605623.75, 605613.1875]","[2.435884714126587, 2.8246657848358154, 2.7655...","[1.0, 25.0, 20.0, 26.0]","[1.0, 0.0, 0.0, 0.0]","[509.0552978515625, 438.989990234375, 448.3741...",2.248323e-07
4,"[685463.3125, 685425.4375, 685578.5625, 685463...","[2.519512414932251, 2.8856987953186035, 2.6414...","[2.0, 29.0, 7.0, 0.0]","[1.0, 0.0, 2.0, 1.0]","[492.1587219238281, 429.7052917480469, 469.437...",1.062943e-07
5,"[408189.34375, 408416.3125, 407976.78125, 4081...","[2.7312569618225098, 2.885037422180176, 2.5056...","[7.0, 10.0, 1.0, 9.0, 9.0, 27.0]","[6.0, 8.0, 3.0, 6.0, 6.0, 4.0]","[454.00341796875, 429.80377197265625, 494.8811...",5.277093e-06
...,...,...,...,...,...,...
1597,"[619917.75, 620180.0625, 619861.6875, 619859.6...","[2.341064929962158, 2.7492988109588623, 2.7163...","[13.0, 22.0, 19.0, 25.0]","[1.0, 3.0, 0.0, 0.0]","[529.6734619140625, 451.0240783691406, 456.493...",1.207747e-10
1597,"[619950.5, 619876.1875]","[2.3014776706695557, 2.672804355621338]","[2.0, 15.0]","[1.0, 1.0]","[538.7843017578125, 463.93218994140625]",1.207747e-10
1597,"[619914.0, 619839.125, 619879.0, 619878.6875, ...","[2.7134034633636475, 2.805280923843384, 2.4971...","[11.0, 29.0, 2.0, 8.0, 20.0, 20.0]","[1.0, 0.0, 1.0, 1.0, 0.0, 2.0]","[456.9906311035156, 442.0234680175781, 496.564...",1.207747e-10
